In [102]:
import os
import xml.etree.ElementTree as ET
import json
import pickle as pkl

In [8]:
folder = "/usrvol/data/RTE1"

In [9]:
files = os.listdir(folder)

In [67]:
def process_RTE(folder):
    data_list = []
    filename_list = []
    files = os.listdir(folder)
    for file in files:
        if file.endswith("xml"):
            data = []
            filename = file.split('.')[0]
            with open(f"{folder}/{file}", "r") as f:
                tree = ET.parse(f)
            root = tree.getroot()
            data = []
            for child in root:
                for child2 in child:
                    for element in child2.iter('t'):
                        sentence1 = element.text
                    for element in child2.iter('h'):
                        sentence2 = element.text
                info = {'sentence1': sentence1, 'sentence2': sentence2, 
                        'label': child.attrib['value'], 'id': child.attrib['id'],
                        'task': child.attrib['task']}
                data.append(info)
            data_list.append(data)
            filename_list.append(filename)
    return data_list, filename_list
    

In [68]:
data_list, filename_list = process_RTE(folder)

In [77]:
filename_list[2]

'dev2'

In [78]:
folder = "/usrvol/data/SciTail"

In [82]:
os.listdir(folder)

['scitail_1.0_dev.txt', 'scitail_1.0_train.txt', 'scitail_1.0_test.txt']

In [88]:
def processSciTail(folder):
    files = os.listdir(folder)
    data_list = []
    filename_list = []
    for file in files:
        if file.endswith("txt"):
            data = []
            filename = file.split('_')[2].split('.')[0]
            with open(f"{folder}/{file}", 'r') as f:
                for line in f:
                    data.append(json.loads(line))
            data_list.append(data)
            filename_list.append(filename)
    return data_list, filename_list

In [89]:
data_list, filename_list = processSciTail(folder)

In [101]:
data_list[1][-1]

{'sentence1_binary_parse': '( ( This expansion ) ( ( ( ( causes ( the light ) ) ( from ( distant stars ) ) ) ( to ( be ( shifted ( towards ( ( the ( red end ) ) ( of ( the spectrum ) ) ) ) ) ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (DT This) (NN expansion)) (VP (VBZ causes) (NP (DT the) (NN light)) (PP (IN from) (NP (JJ distant) (NNS stars))) (S (VP (TO to) (VP (VB be) (VP (VBN shifted) (PP (IN towards) (NP (NP (DT the) (JJ red) (NN end)) (PP (IN of) (NP (DT the) (NN spectrum)))))))))) (. .)))',
 'gold_label': 'entailment',
 'sentence2_binary_parse': '( Red-shift ( ( ( refers ( to ( ( a shift ) ( toward red ) ) ) ) ( in ( ( the spectrum ) ( from stars ) ) ) ) . ) )',
 'sentence1': 'This expansion causes the light from distant stars to be shifted towards the red end of the spectrum.',
 'annotator_labels': ['entailment',
  'entailment',
  'entailment',
  'entailment',
  '-'],
 'sentence2_parse': '(ROOT (S (NP (NNP Red-shift)) (VP (VBZ refers) (PP (TO to) (NP (NP (DT a) (NN shift))

In [103]:
with open("/usrvol/processed_data/RTE1/dev/sintactic0.pkl", "rb") as f:
    data = pkl.load(f)

In [104]:
len(data)

287

# For paper images

In [1]:
from supar import Parser
import networkx as nx
import matplotlib.pyplot as plt
import pickle as pkl
import torch

sintactic tree

In [3]:
device = 'cuda:0'
torch.cuda.set_device(device)
model = 'dep-biaffine-roberta-en'
sint = Parser.load(model)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
sentence = 'I saw Sarah with a telescope.'

In [6]:
prediction = sint.predict(sentence, verbose=False, lang='en')

In [10]:
prediction[0]

1	I	_	_	_	_	2	nsubj	_	_
2	saw	_	_	_	_	0	root	_	_
3	Sarah	_	_	_	_	2	dobj	_	_
4	with	_	_	_	_	2	prep	_	_
5	a	_	_	_	_	6	det	_	_
6	telescope	_	_	_	_	4	pobj	_	_
7	.	_	_	_	_	2	punct	_	_

In [12]:
torch.cuda.empty_cache()

semantic tree

In [13]:
device = 'cuda:0'
torch.cuda.set_device(device)
model='sdp-vi-en'
sem = Parser.load(model)

In [ ]:
sentence = 'I saw Sarah with a telescope.'

In [14]:
prediction = sem.predict(sentence, verbose=False, lang='en')

In [15]:
prediction[0]

1	I	_	_	_	_	_	_	2:ARG1	_
2	saw	_	_	_	_	_	_	0:root|2:ARG1	_
3	Sarah	_	_	_	_	_	_	2:ARG2	_
4	with	_	_	_	_	_	_	_	_
5	a	_	_	_	_	_	_	_	_
6	telescope	_	_	_	_	_	_	4:ARG2|5:BV	_
7	.	_	_	_	_	_	_	_	_

In [16]:
torch.cuda.empty_cache()

constituency tree

In [2]:
device = 'cuda:0'
torch.cuda.set_device(device)
model='con-crf-roberta-en'
con = Parser.load(model)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
sentence = 'I saw Sarah with a telescope.'

In [5]:
prediction = con.predict(sentence, verbose=False, lang='en')

In [6]:
prediction.

(TOP (S (NP (_ I)) (VP (_ saw) (NP (_ Sarah)) (PP (_ with) (NP (_ a) (_ telescope)))) (_ .)))